In [1]:
# there are other ways to do this but this is the simplest. feel free to adapt to your needs
# TFE_ADDR =  os.environ.get('TFE_ADDR', 'https://app.terraform.io')
# TFE_API_TOKEN =  os.environ.get('TFE_API_TOKEN', '')
TFE_ADDR = 'https://app.terraform.io'
TFE_API_TOKEN = ''
TFE_ORG = 'emea-se-playground-2019'

In [2]:
# needed imports, this would hopefully be ignored.
import json
import requests
import os
import pprint
import pandas
pp = pprint.PrettyPrinter()

headers = {
    'Content-Type': 'application/vnd.api+json',
    'Authorization': f"Bearer {TFE_API_TOKEN}"  
    }

# I'll define all the needed functions here to make the notebook a bit cleaner

def getWorkspaceInfo(organisation):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        for item in todo_item:
            retorno.append({'id': item['id'],
                            'name': item['attributes']['name'],
                            'created-at': item['attributes']['created-at'],
                            'latest-change-at': item['attributes']['latest-change-at'],
                            'terraform-version': item['attributes']['terraform-version'],
                            'vcs-repo': vcsrepo(item),
                            'current-run': currentruninfo(item),
                            'latest-run': latestruninfo(item),
                            })
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    return retorno

def getRunsInfo(workspace_id):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/workspaces/{workspace_id}/runs?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    todo_item = resp.json()['data']
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        pp.pprint(f"We are in page {current_page} out of {total_pages} for workspace {workspace_id}")
        for item in todo_item:
            retorno.append(item['id'])
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/workspaces/{workspace_id}/runs?page[size]=100&page[number]={page}', headers=headers )
    return retorno

def vcsrepo(item):
    try:
       return  item['attributes']['vcs-repo']['display-identifier'] 
    except TypeError:
       return ''

def currentruninfo(item):
    try:
       link =  item['relationships']['current-run']['links']['related']
       resp = requests.get(TFE_ADDR+f'{link}', headers=headers )
       if resp.status_code != 200:
          pp.pprint(f"something went wrong: {resp}")
          return ""
       ce = ""
       try:
           ce = costestimate(resp.json()['data']['relationships']['cost-estimate']['links']['related'])
       except Exception:
            ce = 'cost estimation failed'
       retorno = {
        'id': resp.json()['data']['id'],
        'status': resp.json()['data']['attributes']['status'],
        "trigger-reason": resp.json()['data']['attributes']["trigger-reason"],
        "applied-at": resp.json()['data']['attributes']['status-timestamps']['applied-at'],
        "planned-at": resp.json()['data']['attributes']['status-timestamps']['planned-at'],
        "applying-at": resp.json()['data']['attributes']['status-timestamps']['applying-at'],
        "planning-at": resp.json()['data']['attributes']['status-timestamps']['planning-at'],
        "confirmed-at": resp.json()['data']['attributes']['status-timestamps']['confirmed-at'],
        "plan-queued-at": resp.json()['data']['attributes']['status-timestamps']['plan-queued-at'],
        "apply-queued-at": resp.json()['data']['attributes']['status-timestamps']['apply-queued-at'],
        "cost-estimated-at": resp.json()['data']['attributes']['status-timestamps']['cost-estimated-at'],
        "plan-queueable-at": resp.json()['data']['attributes']['status-timestamps']['plan-queueable-at'],
        "cost-estimating-at": resp.json()['data']['attributes']['status-timestamps']['cost-estimating-at'],
        "cost-estimate":  ce
        }
       return retorno
    except KeyError:
       return 'failed'

def latestruninfo(item):
    try:
       link =  item['relationships']['latest-run']['links']['related']
       resp = requests.get(TFE_ADDR+f'{link}', headers=headers )
       if resp.status_code != 200:
          pp.pprint(f"something went wrong: {resp}")
          return ""
       ce = ""
       try:
           ce = costestimate(resp.json()['data']['relationships']['cost-estimate']['links']['related'])
       except Exception:
            ce = 'cost estimation failed'
       retorno = {
        'id': resp.json()['data']['id'],
        'status': resp.json()['data']['attributes']['status'],
        "trigger-reason": resp.json()['data']['attributes']["trigger-reason"],
        "applied-at": resp.json()['data']['attributes']['status-timestamps']['applied-at'],
        "planned-at": resp.json()['data']['attributes']['status-timestamps']['planned-at'],
        "applying-at": resp.json()['data']['attributes']['status-timestamps']['applying-at'],
        "planning-at": resp.json()['data']['attributes']['status-timestamps']['planning-at'],
        "confirmed-at": resp.json()['data']['attributes']['status-timestamps']['confirmed-at'],
        "plan-queued-at": resp.json()['data']['attributes']['status-timestamps']['plan-queued-at'],
        "apply-queued-at": resp.json()['data']['attributes']['status-timestamps']['apply-queued-at'],
        "cost-estimated-at": resp.json()['data']['attributes']['status-timestamps']['cost-estimated-at'],
        "plan-queueable-at": resp.json()['data']['attributes']['status-timestamps']['plan-queueable-at'],
        "cost-estimating-at": resp.json()['data']['attributes']['status-timestamps']['cost-estimating-at'],
         "cost-estimate":  ce
        }
       return retorno
    except KeyError:
       return 'failed'

def costestimate(link):
    resp = requests.get(TFE_ADDR+f'{link}', headers=headers )
    if resp.status_code != 200:
       pp.pprint(f"something went wrong: {resp}")
       return resp
    retorno = {
     'id': resp.json()['data']['id'],
     'status': resp.json()['data']['attributes']['status'],
     "queued-at": resp.json()['data']['attributes']['status-timestamps']['queued-at'],
     "finished-at": resp.json()['data']['attributes']['status-timestamps']['finished-at'],
      "resources":   resp.json()['data']['attributes']['resources']
      }
    return retorno

In [6]:
retorno = getWorkspaceInfo(TFE_ORG)

[{'id': 'ws-NNPCZRmKM6AvB1uE',
  'name': 'ricardo-aws-demostack',
  'created-at': '2019-08-08T13:23:00.619Z',
  'latest-change-at': '2020-08-24T18:21:33.053Z',
  'terraform-version': '0.12.26',
  'vcs-repo': '',
  'current-run': {'id': 'run-ToYmYdL9b62HcL3q',
   'status': 'applied',
   'trigger-reason': 'manual',
   'applied-at': '2020-08-24T18:21:34+00:00',
   'planned-at': '2020-08-24T18:14:09+00:00',
   'applying-at': '2020-08-24T18:14:14+00:00',
   'planning-at': '2020-08-24T18:13:26+00:00',
   'confirmed-at': '2020-08-24T18:14:14+00:00',
   'plan-queued-at': '2020-08-24T18:13:25+00:00',
   'apply-queued-at': '2020-08-24T18:14:14+00:00',
   'cost-estimated-at': '2020-08-24T18:14:13+00:00',
   'plan-queueable-at': '2020-08-24T18:13:25+00:00',
   'cost-estimating-at': '2020-08-24T18:14:09+00:00',
   'cost-estimate': {'id': 'ce-vrM8eH89bGD3bM2J',
    'status': 'finished',
    'queued-at': '2020-08-24T18:14:10+00:00',
    'finished-at': '2020-08-24T18:14:13+00:00',
    'resources': {'m

In [7]:
 data = pandas.DataFrame(retorno)

In [11]:
cost = data['current-run']
cost[0]

{'id': 'run-ToYmYdL9b62HcL3q',
 'status': 'applied',
 'trigger-reason': 'manual',
 'applied-at': '2020-08-24T18:21:34+00:00',
 'planned-at': '2020-08-24T18:14:09+00:00',
 'applying-at': '2020-08-24T18:14:14+00:00',
 'planning-at': '2020-08-24T18:13:26+00:00',
 'confirmed-at': '2020-08-24T18:14:14+00:00',
 'plan-queued-at': '2020-08-24T18:13:25+00:00',
 'apply-queued-at': '2020-08-24T18:14:14+00:00',
 'cost-estimated-at': '2020-08-24T18:14:13+00:00',
 'plan-queueable-at': '2020-08-24T18:13:25+00:00',
 'cost-estimating-at': '2020-08-24T18:14:09+00:00',
 'cost-estimate': {'id': 'ce-vrM8eH89bGD3bM2J',
  'status': 'finished',
  'queued-at': '2020-08-24T18:14:10+00:00',
  'finished-at': '2020-08-24T18:14:13+00:00',
  'resources': {'matched': [{'address': 'module.primarycluster.aws_alb.nomad',
     'module': 'module.primarycluster',
     'name': 'nomad',
     'provider': 'aws',
     'type': 'aws_alb',
     'hourly-cost': '0.0',
     'prior-monthly-cost': '16.2',
     'proposed-monthly-cost': 